In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load dataset
data = pd.read_csv("hf://datasets/stepp1/tweet_emotion_intensity/train.csv")

# Split dataset into training, validation, and test sets
train_data, temp_data = train_test_split(data, test_size=0.3, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

print(f"Training set size: {len(train_data)}")
print(f"Validation set size: {len(val_data)}")
print(f"Test set size: {len(test_data)}")

In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

# Load pre-trained BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results_traditional',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    eval_strategy="epoch",
)

# Fine-tune the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
)

# Start fine-tuning
trainer.train()

In [ ]:
from lora import LoRALayer

# Apply LoRA to specific layers (e.g., attention layers)
for name, module in model.named_modules():
    if 'attention' in name:
        module.apply(LoRALayer)

# Freeze the rest of the model
for param in model.base_model.parameters():
    param.requires_grad = False

# Fine-tune the LoRA-enhanced model
trainer.train()

In [ ]:
from qlora import QuantizeModel

# Quantize the model to reduce memory usage
quantized_model = QuantizeModel(model, bits=8)

# Apply LoRA to specific layers in the quantized model
for name, module in quantized_model.named_modules():
    if 'attention' in name:
        module.apply(LoRALayer)

# Fine-tune the QLoRA-enhanced model
trainer.train()